<a href="https://colab.research.google.com/github/jihokwak/mlstudy/blob/master/keras/%EC%BC%80%EB%9D%BC%EC%8A%A4%EC%B0%BD%EC%8B%9C%EC%9E%90_%EA%B3%A0%EA%B8%89%EB%8F%84%EA%B5%AC_%ED%95%A8%EC%88%98%ED%98%95API__%EC%97%B0%EC%8A%B53_%EB%8B%A4%EC%A4%91%EC%B6%9C%EB%A0%A5%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras import layers
from keras import Input
from keras.models import Model

Using TensorFlow backend.


In [0]:
vocabulary_size = 50000

In [0]:
num_income_groups = 10

In [0]:
posts_input = Input(shape=(None,), dtype='int32', name='posts')

In [5]:
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x

<tf.Tensor 'conv1d_2/Relu:0' shape=(?, ?, 128) dtype=float32>

In [10]:
x = layers.MaxPooling1D(5)(x)
x

<tf.Tensor 'max_pooling1d_3/Squeeze:0' shape=(?, ?, 128) dtype=float32>

In [11]:
x = layers.Conv1D(256, 5, activation='relu')(x)
x

<tf.Tensor 'conv1d_3/Relu:0' shape=(?, ?, 256) dtype=float32>

In [12]:
x = layers.Conv1D(256, 5, activation='relu')(x)
x

<tf.Tensor 'conv1d_4/Relu:0' shape=(?, ?, 256) dtype=float32>

In [13]:
x = layers.MaxPooling1D(5)(x)
x

<tf.Tensor 'max_pooling1d_4/Squeeze:0' shape=(?, ?, 256) dtype=float32>

In [14]:
x = layers.Conv1D(256, 5, activation='relu')(x)
x

<tf.Tensor 'conv1d_5/Relu:0' shape=(?, ?, 256) dtype=float32>

In [15]:
x = layers.Conv1D(256, 5, activation='relu')(x)
x

<tf.Tensor 'conv1d_6/Relu:0' shape=(?, ?, 256) dtype=float32>

In [16]:
x = layers.GlobalMaxPooling1D()(x)
x

<tf.Tensor 'global_max_pooling1d_1/Max:0' shape=(?, 256) dtype=float32>

In [17]:
x = layers.Dense(128, activation='relu')(x)
x

<tf.Tensor 'dense_1/Relu:0' shape=(?, 128) dtype=float32>

In [0]:
age_prediction = layers.Dense(1, name='age')(x) #회귀문제
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x) #다중분류문제
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x) #이진분류문제

In [0]:
model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

In [20]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, None, 128)    163968      embedding_1[0][0]                
__________________________________________________________________________________________________
max_pooling1d_3 (MaxPooling1D)  (None, None, 128)    0           conv1d_2[0][0]                   
__________________________________________________________________________________________________
conv1d_3 (

In [0]:
model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

In [0]:
#이름지정 방식(위와 동일)
model.compile(optimizer='rmsprop', loss={
    'age': 'mse',
    'income':'categorical_crossentropy',
    'gender':'binary_crossentropy'
})

In [0]:
#개별손실값의 최종손실 기여수준 지정

In [0]:
model.compile(optimizer='rmsprop',
             loss = {
                  'age': 'mse',
                  'income':'categorical_crossentropy',
                  'gender':'binary_crossentropy'
             },
             loss_weights={
                 'age' : 0.25,
                 'income': 1.,
                 'gender': 10.
             })

In [0]:
model.fit(posts_input, 
         {
             'age': age_targets,
             'income':income_targets,
             'gender':gender_targets
         }, 
          epochs= 10, batch_size=64 )